Jan 11, 2021

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

from pyspark.sql.functions import when, udf, col, regexp_extract, regexp_replace,count,sum,avg,round
from pyspark.sql.types import DoubleType,IntegerType, StringType

# 스파크 통계
import pyspark.sql.functions as F

In [2]:
sc = SparkContext( 'local' ) 
sqlCtx = SQLContext( sc )

In [4]:
data1 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Name': {0: 'Owen', 1: 'Florence', 2: 'Laina', 3: 'Lily', 4: 'William'},
         'Sex': {0: 'male', 1: 'female', 2: 'female', 3: 'female', 4: 'male'},
         'Survived': {0: 0, 1: 1, 2: 1, 3: 1, 4: 0}}

data2 = {'PassengerId': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
         'Age': {0: 22, 1: 38, 2: 26, 3: 35, 4: 35},
         'Fare': {0: 7.3, 1: 71.3, 2: 7.9, 3: 53.1, 4: 8.0},
         'Pclass': {0: 3, 1: 1, 2: 3, 3: 1, 4: 3}}

In [6]:
# 판다스 데이터 프레임
df1_pd = pd.DataFrame( data1)
df2_pd = pd.DataFrame( data2)

In [7]:
df1_pd

,PassengerId,Name,Sex,Survived
0,1,Owen,male,0
1,2,Florence,female,1
2,3,Laina,female,1
3,4,Lily,female,1
4,5,William,male,0


In [8]:
df2_pd

,PassengerId,Age,Fare,Pclass
0,1,22,7.3,3
1,2,38,71.3,1
2,3,26,7.9,3
3,4,35,53.1,1
4,5,35,8.0,3


In [9]:
# 스파크 데이터 프레임
df1 = sqlCtx.createDataFrame(df1_pd)
df2 = sqlCtx.createDataFrame(df2_pd)

In [10]:
df1.show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
+-----------+--------+------+--------+



In [11]:
df2.show()

+-----------+---+----+------+
|PassengerId|Age|Fare|Pclass|
+-----------+---+----+------+
|          1| 22| 7.3|     3|
|          2| 38|71.3|     1|
|          3| 26| 7.9|     3|
|          4| 35|53.1|     1|
|          5| 35| 8.0|     3|
+-----------+---+----+------+



In [13]:
df1.join( df2, on=['PassengerId'] ).show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          5| William|  male|       0| 35| 8.0|     3|
|          1|    Owen|  male|       0| 22| 7.3|     3|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          2|Florence|female|       1| 38|71.3|     1|
|          4|    Lily|female|       1| 35|53.1|     1|
+-----------+--------+------+--------+---+----+------+



# 연습문제

In [ ]:
1.df1 과 동일한 데이터를 row로 추가 하시요
2.PassengerId 기준으로 join 하시요
#아래문제는 join 데이터를 이용하여 ...
-------------
3. 등급별 나이와 금액의 평균을 구하시요
4. 등급별 갯수와, 나이평균, 금액의 합을 구하시요
5. 가격이 높은 순으로 정렬하시요

In [16]:
# 1.df1 과 동일한 데이터를 row로 추가 하시요
df1.unionAll( df1 ).show()

+-----------+--------+------+--------+
|PassengerId|    Name|   Sex|Survived|
+-----------+--------+------+--------+
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
|          1|    Owen|  male|       0|
|          2|Florence|female|       1|
|          3|   Laina|female|       1|
|          4|    Lily|female|       1|
|          5| William|  male|       0|
+-----------+--------+------+--------+



In [50]:
# 2. assengerId 기준으로 join 하시요
df3 = df1.join( df2, on=['PassengerId'] ).orderBy(['PassengerId'] )
df3.show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          1|    Owen|  male|       0| 22| 7.3|     3|
|          2|Florence|female|       1| 38|71.3|     1|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          4|    Lily|female|       1| 35|53.1|     1|
|          5| William|  male|       0| 35| 8.0|     3|
+-----------+--------+------+--------+---+----+------+



In [49]:
# 3. 등급별 나이와 금액의 평균을 구하시요
df3.groupBy(['Pclass'] ).\
    agg( F.mean( df3['Age']).alias('평균 나이'),  F.mean( df3['Fare']).alias('평균 금액') ).\
    show()

+------+------------------+-----------------+
|Pclass|         평균 나이|        평균 금액|
+------+------------------+-----------------+
|     1|              36.5|             62.2|
|     3|27.666666666666668|7.733333333333333|
+------+------------------+-----------------+



In [41]:
# 4. 등급별 갯수와, 나이평균, 금액의 합을 구하시요
df3.groupBy(['Pclass'] ).agg(   F.count( df3['Name'] ).alias('등급별 갯수'),
                                round( F.mean( df3['Age']), 2).alias('평균 나이'),
                                round( F.sum( df3['Fare']), 2).alias('금액의 합') 
                            ).show()

+------+-----------+---------+---------+
|Pclass|등급별 갯수|평균 나이|금액의 합|
+------+-----------+---------+---------+
|     1|          2|     36.5|    124.4|
|     3|          3|    27.67|     23.2|
+------+-----------+---------+---------+



In [51]:
# pclass기준으로 묶고 pclass기준으로 count해도 된다.
df3.groupBy(['Pclass'] ).agg(   F.count( df3['Pclass'] ).alias('등급별 갯수'),
                                round( F.mean( df3['Age']), 2).alias('평균 나이'),
                                round( F.sum( df3['Fare']), 2).alias('금액의 합') 
                            ).show()

+------+-----------+---------+---------+
|Pclass|등급별 갯수|평균 나이|금액의 합|
+------+-----------+---------+---------+
|     1|          2|     36.5|    124.4|
|     3|          3|    27.67|     23.2|
+------+-----------+---------+---------+



In [30]:
# 5. 가격이 높은 순으로 정렬하시요
df3.orderBy( df3['Fare'], ascending=False ).show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          2|Florence|female|       1| 38|71.3|     1|
|          4|    Lily|female|       1| 35|53.1|     1|
|          5| William|  male|       0| 35| 8.0|     3|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          1|    Owen|  male|       0| 22| 7.3|     3|
+-----------+--------+------+--------+---+----+------+



In [47]:
df3.sort( 'Fare' , ascending=False ).show()

+-----------+--------+------+--------+---+----+------+
|PassengerId|    Name|   Sex|Survived|Age|Fare|Pclass|
+-----------+--------+------+--------+---+----+------+
|          2|Florence|female|       1| 38|71.3|     1|
|          4|    Lily|female|       1| 35|53.1|     1|
|          5| William|  male|       0| 35| 8.0|     3|
|          3|   Laina|female|       1| 26| 7.9|     3|
|          1|    Owen|  male|       0| 22| 7.3|     3|
+-----------+--------+------+--------+---+----+------+

